In [1]:
import pickle
import pandas as pd 
import numpy as np

## predict的姓名，依據該姓名去取前後文，並且換成李家賢，加入tags=0/1，端看是否為黑名單

In [2]:
unpickled_df = pd.read_pickle("step2_1_input_train_full_data.pkl")# pd.read_pickle("train_full_data.pkl")

In [3]:
unpickled_df.head()#iloc[154:196,:]

,Unnamed: 0,hyperlink,name,article,name_list,article_split,all_name
0,0,https://news.cnyes.com/news/id/4352432,[],理財基金量化交易追求絕對報酬 有效對抗牛熊市鉅亨網記者 鄭心芸2019/07/05 22:3...,[],[理財基金量化交易追求絕對報酬 有效對抗牛熊市鉅亨網記者 鄭心芸2019/07/05 22:...,"[鄭心芸, 張堯勇, 詹姆斯·西蒙斯, 巴菲特]"
1,1,https://udn.com/news/story/120775/4112519,[],10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件，北市府勞動局認定業者未依職業...,[],[10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件，北市府勞動局認定業者未依職...,"[賴香伶, 康水順]"
2,2,https://www.mirrormedia.mg/story/20191008soc011/,[],社會2019.10.08 09:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從...,[],[社會2019.10.08 09:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何...,"[李育材, 德唐, 張慶輝, 君鴻, 凱德唐, 李日順]"
3,3,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,[],例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[],[例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進...,"[沈君融, 陳世恒]"
4,4,http://www.coolloud.org.tw/node/93356,[],內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區，將於8月30日到高雄大社現勘（註...,[],[內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區，將於8月30日到高雄大社現勘（...,"[張勝富, 黃捷, 韓國瑜, 江炳坤, 王敏玲, 吳忠穎, 陳其邁, 邱花妹, 余陳月英, ..."


In [5]:
# 找出字串中，屬於該名字patretn的人
def find_all_indexes(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

#示範
s =unpickled_df.iloc[195].article
print(find_all_indexes(s, '陳淳伍'))

[21, 154, 188, 221, 271, 354, 438, 473, 508, 577]


In [31]:
#need 
article_truncate = [] # 依據名字擷取的文章
article_index = []  # 屬於哪篇文章的index
article_tags = [] # 該人是否是黑名單
article_tags_name = [] # 該人是誰
for i in range(len(unpickled_df)):
    article_len = len(unpickled_df.iloc[i].article)#.article_len
    #文章內文
    article_content = unpickled_df.iloc[i].article
    # 找出共有幾個人
    all_name_list = unpickled_df.iloc[i].all_name
    for j in all_name_list: # 第i篇文章的第j個人
        name_index = find_all_indexes(article_content, j )#出現在文章的index
        for k in name_index:
            if j in unpickled_df.iloc[i].name_list :
                article_tags.append(1)
            else:
                article_tags.append(0)
            article_tags_name.append(j)
            txt = unpickled_df.iloc[i].article[  np.clip(k-min(100 , int(article_len*0.2))
                                             , 0 ,999999
                                                  )  : 
                                  np.clip(k-min(50 , int(article_len*0.2))
                                             , 0 , 99999
                                         )+290
                                        ].replace( j , '李家賢') 
            article_truncate.append(txt)
            article_index.append(i)

In [18]:
lq= []
for i in range(len(article_truncate)):
    lq.append(len(article_truncate[i]))

In [19]:
np.mean(lq)

299.57425775752296

In [20]:
# i= 195
# article_truncate = [] # 依據名字擷取的文章
# article_index = []  # 屬於哪篇文章的index
# article_tags = [] # 該人是否是黑名單
# article_tags_name = [] # 該人是誰
# article_len = unpickled_df.iloc[i].article_len
# #文章內文
# article_content = unpickled_df.iloc[i].article
# # 找出共有幾個人
# all_name_list = unpickled_df.iloc[i].all_name
# for j in all_name_list: # 第i篇文章的第j個人
#     name_index = find_all_indexes(article_content, j )#出現在文章的index
#     for k in name_index:
#         if j in unpickled_df.iloc[i].name_list :
#             article_tags.append(1)
#         else:
#             article_tags.append(0)
#         article_tags_name.append(j)
#         txt = unpickled_df.iloc[i].article[  np.clip(k-min(50 , int(article_len*0.2))
#                                          , 0 ,999999
#                                               )  : 
#                               np.clip(k-min(50 , int(article_len*0.2))
#                                          , 0 , 99999
#                                      )+100
#                                     ].replace( j , '李家賢') 
#         article_truncate.append(txt)
#         article_index.append(i)

In [32]:
df = pd.DataFrame(data={'article_truncate':article_truncate,
                  'article_index':article_index ,
                  'article_tags':article_tags
                  ,'article_tags_name':article_tags_name})

In [33]:
df[df.article_tags==1]

,article_truncate,article_index,article_tags,article_tags_name
109,自稱房產幽默大師的李家賢，涉吸金捲款25億落跑！他自稱炒房專家，在全台授課分享理財，卻遭指控...,15,1,王派宏
110,自稱房產幽默大師的李家賢，涉吸金捲款25億落跑！他自稱炒房專家，在全台授課分享理財，卻遭指控...,15,1,王派宏
111,理財，卻遭指控，4月28號突然聯繫不上，也沒上課，顧問公司以為他發生危險立刻報案，但人卻已經...,15,1,王派宏
112,經捲款，急忙報警追人。自製youtube頻道，還自稱是房產幽默大師，他是李家賢，不只在網路分...,15,1,王派宏
113,3年都ok，都有如期拿到利息，的確有固定利息，從小金額100多萬，到後面越投越多，老師也一直...,15,1,王派宏
...,...,...,...,...
29638,也在台灣地區，並無任何海外資產，他還因病須固定到醫院回診，沒有不返台的可能，並提出桃園市議會...,4563,1,邱佳亮
29639,岸交流促進會參訪行程表（3月7日至3月11日）等為證據，請求暫時解除限制出境。高院法官調查，...,4563,1,邱佳亮
29640,00萬元交保及限制出境；一審認李家賢確實詐領補助費，判他有期徒刑12年6月，併科罰金120萬...,4563,1,邱佳亮
29722,前立委李家賢被控涉入機場捷運BOT案仲裁弊案，事後國稅局認為郭收受7000萬，涉逃稅加罰鍰迄...,4578,1,郭政權


In [34]:
len(df.iloc[0].article_truncate)

290

In [36]:
df.to_csv('step2_2_output_train_data_300.csv' , index=0)

In [ ]:
# 以下測試

In [ ]:
#df.to_csv('step2_train_data_300.csv' , index=0)

In [106]:
df.to_csv('step2_train_data.csv' , index=0)

In [23]:
df.to_csv('step2_train_data_300.csv' , index=0)

In [13]:
df2 = pd.read_csv('step2_train_data_300.csv')

In [14]:
df2[df2.article_tags==1]

,article_truncate,article_index,article_tags,article_tags_name
109,自稱房產幽默大師的李家賢，涉吸金捲款25億落跑！他自稱炒房專家，在全台授課分享理財，卻遭指控...,15,1,王派宏
110,自稱房產幽默大師的李家賢，涉吸金捲款25億落跑！他自稱炒房專家，在全台授課分享理財，卻遭指控...,15,1,王派宏
111,理財，卻遭指控，4月28號突然聯繫不上，也沒上課，顧問公司以為他發生危險立刻報案，但人卻已經...,15,1,王派宏
112,經捲款，急忙報警追人。自製youtube頻道，還自稱是房產幽默大師，他是李家賢，不只在網路分...,15,1,王派宏
113,3年都ok，都有如期拿到利息，的確有固定利息，從小金額100多萬，到後面越投越多，老師也一直...,15,1,王派宏
...,...,...,...,...
29638,也在台灣地區，並無任何海外資產，他還因病須固定到醫院回診，沒有不返台的可能，並提出桃園市議會...,4563,1,邱佳亮
29639,岸交流促進會參訪行程表（3月7日至3月11日）等為證據，請求暫時解除限制出境。高院法官調查，...,4563,1,邱佳亮
29640,00萬元交保及限制出境；一審認李家賢確實詐領補助費，判他有期徒刑12年6月，併科罰金120萬...,4563,1,邱佳亮
29724,前立委李家賢被控涉入機場捷運BOT案仲裁弊案，事後國稅局認為郭收受7000萬，涉逃稅加罰鍰迄...,4578,1,郭政權


In [83]:
unpickled_df.iloc[195].name_list                                  

['陳淳伍']

In [91]:
min(np.array([50,10]))

10